In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
df_pysynthlab = pd.read_csv('../benchmark/sygus_solver_pysynthlab_results_new.csv')
df_pysynthlab['run_datetime'] = pd.to_datetime(df_pysynthlab['run_datetime'])
df_pysynthlab['solver'] = 'PySynthLab'
df_pysynthlab['config'] = df_pysynthlab['config'].str.split('/').str[-1].str.replace('.yaml', '').str.replace('benchmark_','')
df_pysynthlab['file'] = df_pysynthlab['file'].str.split('/').str[-1].str.replace('.sl', '')
df_pysynthlab.rename(columns={'file': 'problem'}, inplace=True)
df_pysynthlab['success'] = df_pysynthlab['return_code'] == 0

def extract_solution(stdout, return_code):
    if return_code == 0 and 'Metrics summary:' in stdout:
        lines = stdout.strip().split('\n')
        if len(lines) > 1:
            res = lines[1].split(': ', 1)[1] if ': ' in lines[1] else lines[1]
            return res.strip()
    return ''
    
df_pysynthlab['solution'] = df_pysynthlab.apply(lambda row: extract_solution(row['stdout'], row['return_code']), axis=1)
df_pysynthlab.head()

,run_id,run_datetime,solver,config,problem,return_code,time,avg_cpu_percentage,max_memory_usage,avg_all_cpu,...,solution_complexity,best_partial_score,grammar_size,solution_found,solution_space_coverage,solver_calls,unique_patterns,pattern_reuse_ratio,success,solution
0,92e6b38a-682d-4011-9951-02bbe8cbfb02,2024-08-12 21:44:47.838685,PySynthLab,random_enumerative_bottom_up,absolute,0,1.002077,12.483333,0.000000,"[13.3, 8.1, 16.8, 8.1, 28.6, 8.9, 20.4, 12.0, ...",...,1.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True,Var(0)*-1
1,92e6b38a-682d-4011-9951-02bbe8cbfb02,2024-08-12 21:44:47.838685,PySynthLab,fast_enumerative,absolute,0,1.002610,13.312500,0.000000,"[16.2, 9.0, 15.2, 8.1, 29.0, 8.1, 23.8, 11.0, ...",...,0.0,0.0,24.0,True,0.0,0.0,0.0,0.0,True,Var(0)
2,37082fdc-3fe2-4b9a-94f7-14b51a869864,2024-08-12 21:46:05.526144,PySynthLab,random_enumerative_bottom_up,absolute,0,1.002075,13.150000,0.000000,"[16.7, 8.9, 11.5, 9.0, 22.7, 23.5, 17.5, 8.0, ...",...,1.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True,Var(0)*-1
3,37082fdc-3fe2-4b9a-94f7-14b51a869864,2024-08-12 21:46:05.526144,PySynthLab,fast_enumerative,absolute,0,1.003117,12.750000,0.000000,"[13.0, 10.0, 14.6, 9.0, 24.7, 8.9, 18.6, 19.0,...",...,0.0,0.0,24.0,True,0.0,0.0,0.0,0.0,True,Var(0)
4,37082fdc-3fe2-4b9a-94f7-14b51a869864,2024-08-12 21:46:05.526144,PySynthLab,random_weighted_top_down,absolute,-15,30.020212,9.700972,88.308594,"[7.2, 5.8933333333333335, 6.213333333333334, 5...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,


In [18]:
df_cvc5 = pd.read_csv('../benchmark/sygus_solver_cvc5_results_new.csv')
df_cvc5['config'] = (df_cvc5['config']
                .str.replace('--tlimit=30000', '')
                .str.replace('--sygus-enum=', '')
                .str.replace('--sygus-si=', '_single_invocation_')
                .str.replace('--', '_')
                .str.replace('-', '_')
                .str.replace('=', '_')
                .str.replace(' ', '')
                )
df_cvc5['file'] = df_cvc5['file'].str.replace('/data/Documents/DataScience/Courses/Dissertation/problems/sygus_comp_2019_clia_track/', '').str.replace('.sl','').str.strip()
df_cvc5.rename(columns={'file': 'problem'}, inplace=True)
df_cvc5['solver'] = 'CVC5'
df_cvc5['success'] = df_cvc5['return_code'] == 0
df_cvc5['solution'] = df_cvc5['stdout'].apply(lambda x: x.strip().replace('\n','') if pd.notna(x) else '')
df_cvc5.head()

,run_id,run_datetime,solver,config,problem,return_code,time,avg_cpu_usage,max_memory_usage,stdout,stderr,success,solution
0,b699c394-9c7c-489d-ad5c-9ca3fb146a88,2024-08-11T16:52:48.609846,CVC5,smart,absolute,0,0.100908,0.0,0.382812,(\n(define-fun abs ((x Int)) Int 1)\n)\n,NaN,True,((define-fun abs ((x Int)) Int 1))
1,b699c394-9c7c-489d-ad5c-9ca3fb146a88,2024-08-11T16:52:48.609846,CVC5,random,absolute,0,0.101807,0.0,0.421875,(\n(define-fun abs ((x Int)) Int x)\n)\n,NaN,True,((define-fun abs ((x Int)) Int x))
2,b699c394-9c7c-489d-ad5c-9ca3fb146a88,2024-08-11T16:52:48.609846,CVC5,fast,absolute,0,0.101644,0.0,0.222656,(\n(define-fun abs ((x Int)) Int x)\n)\n,NaN,True,((define-fun abs ((x Int)) Int x))
3,b699c394-9c7c-489d-ad5c-9ca3fb146a88,2024-08-11T16:52:48.609846,CVC5,var_agnostic,absolute,0,0.101853,0.0,0.421875,(\n(define-fun abs ((x Int)) Int 1)\n)\n,NaN,True,((define-fun abs ((x Int)) Int 1))
4,b699c394-9c7c-489d-ad5c-9ca3fb146a88,2024-08-11T16:52:48.609846,CVC5,smart_single_invocation_none,absolute,0,0.101616,0.0,0.195312,(\n(define-fun abs ((x Int)) Int 1)\n)\n,NaN,True,((define-fun abs ((x Int)) Int 1))


In [ ]:
df_comp = pd.read_csv('../../other_solvers/sygus_comp_2017_07_03_results/job_info.csv')
df_comp.head()

,run_id,run_datetime,solver,config,file,return_code,time,avg_cpu_percentage,max_memory_usage,avg_all_cpu,...,solution_complexity,best_partial_score,grammar_size,solution_found,solution_space_coverage,solver_calls,unique_patterns,pattern_reuse_ratio,success,solution
0,92e6b38a-682d-4011-9951-02bbe8cbfb02,2024-08-12 21:44:47.838685,PySynthLab,random_enumerative_bottom_up,absolute,0,1.002077,12.483333,0.000000,"[13.3, 8.1, 16.8, 8.1, 28.6, 8.9, 20.4, 12.0, ...",...,1.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True,"Metrics summary: {'candidates_generated': 2, '..."
1,92e6b38a-682d-4011-9951-02bbe8cbfb02,2024-08-12 21:44:47.838685,PySynthLab,fast_enumerative,absolute,0,1.002610,13.312500,0.000000,"[16.2, 9.0, 15.2, 8.1, 29.0, 8.1, 23.8, 11.0, ...",...,0.0,0.0,24.0,True,0.0,0.0,0.0,0.0,True,"Metrics summary: {'candidates_generated': 1, '..."
2,37082fdc-3fe2-4b9a-94f7-14b51a869864,2024-08-12 21:46:05.526144,PySynthLab,random_enumerative_bottom_up,absolute,0,1.002075,13.150000,0.000000,"[16.7, 8.9, 11.5, 9.0, 22.7, 23.5, 17.5, 8.0, ...",...,1.0,0.0,0.0,True,0.0,0.0,0.0,0.0,True,"Metrics summary: {'candidates_generated': 2, '..."
3,37082fdc-3fe2-4b9a-94f7-14b51a869864,2024-08-12 21:46:05.526144,PySynthLab,fast_enumerative,absolute,0,1.003117,12.750000,0.000000,"[13.0, 10.0, 14.6, 9.0, 24.7, 8.9, 18.6, 19.0,...",...,0.0,0.0,24.0,True,0.0,0.0,0.0,0.0,True,"Metrics summary: {'candidates_generated': 1, '..."
4,37082fdc-3fe2-4b9a-94f7-14b51a869864,2024-08-12 21:46:05.526144,PySynthLab,random_weighted_top_down,absolute,-15,30.020212,9.700972,88.308594,"[7.2, 5.8933333333333335, 6.213333333333334, 5...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,
